# Imports

In [1]:
from pathlib import Path

#    Data Manipulation
import pandas as pd
#    pandas settings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#    Linear Algebra
import numpy as np

#    Data Visualization
import matplotlib.pyplot as plt    
import seaborn as sns

#    Regular Expressions
import re

# Load Data

In [2]:
raw_data = '../data/01-raw/911_Calls_For_Service.csv'
data_path = Path(raw_data).absolute()

df = pd.read_csv(data_path)
df.head(3)

/home/btr-devel/.cache/pypoetry/virtualenvs/predictive911-u3V24gzk-py3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,12,13,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,incident_id,agency,incident_address,zip_code,priority,callcode,calldescription,category,call_timestamp,precinct_sca,respondingunit,officerinitiated,intaketime,dispatchtime,traveltime,totalresponsetime,time_on_scene,totaltime,neighborhood,block_id,council_district,longitude,latitude,oid
0,201626400002,DPD,Promenade Ave & Roseberry St,48213,3,825030,DISTURBANCE,DISTURB,2016/09/20 09:36:12+00,910A,NaN,No,2.2,528.6,15.5,546.6,120.5,667.1,Wade,2.616350e+14,4.0,-82.986962,42.408358,455412440
1,201626400019,DPD,Ardmore St & W 7 Mile Rd,48235,2,811020,AUTO X UNK INJ / IMPAIRED,ACCUNK,2016/09/20 09:46:32+00,1201,NaN,No,2.7,5.2,6.4,14.5,59.1,73.6,Tri-Point,2.616354e+14,2.0,-83.185213,42.430914,455412441
2,201626400020,DPD,Pembroke Ave & Prairie St,48221,3,374030,RECOVER AUTO,RECAUTO,2016/09/20 09:48:24+00,123B,NaN,No,4.9,NaN,NaN,NaN,NaN,NaN,Oak Grove,2.616354e+14,2.0,-83.146057,42.438770,455412442


# Explore

In [3]:
df.head()

,incident_id,agency,incident_address,zip_code,priority,callcode,calldescription,category,call_timestamp,precinct_sca,respondingunit,officerinitiated,intaketime,dispatchtime,traveltime,totalresponsetime,time_on_scene,totaltime,neighborhood,block_id,council_district,longitude,latitude,oid
0,201626400002,DPD,Promenade Ave & Roseberry St,48213,3,825030,DISTURBANCE,DISTURB,2016/09/20 09:36:12+00,910A,NaN,No,2.2,528.6,15.5,546.6,120.5,667.1,Wade,2.616350e+14,4.0,-82.986962,42.408358,455412440
1,201626400019,DPD,Ardmore St & W 7 Mile Rd,48235,2,811020,AUTO X UNK INJ / IMPAIRED,ACCUNK,2016/09/20 09:46:32+00,1201,NaN,No,2.7,5.2,6.4,14.5,59.1,73.6,Tri-Point,2.616354e+14,2.0,-83.185213,42.430914,455412441
2,201626400020,DPD,Pembroke Ave & Prairie St,48221,3,374030,RECOVER AUTO,RECAUTO,2016/09/20 09:48:24+00,123B,NaN,No,4.9,NaN,NaN,NaN,NaN,NaN,Oak Grove,2.616354e+14,2.0,-83.146057,42.438770,455412442
3,201626400024,DPD,Joy Rd & Robson St,48228,3,825030,DISTURBANCE,DISTURB,2016/09/20 09:50:29+00,0207,NaN,No,1.4,3.4,7.6,15.0,41.7,56.7,We Care Community,2.616354e+14,7.0,-83.191603,42.358257,455412443
4,201626400029,DPD,Littlefield St & Ellis St,48228,3,374030,RECOVER AUTO,RECAUTO,2016/09/20 09:54:37+00,28A,NaN,No,3,"1,645.3",NaN,NaN,NaN,"1,668.4",Barton-McFarland,2.616354e+14,7.0,-83.175124,42.360867,455412444


In [4]:
df.tail()

,incident_id,agency,incident_address,zip_code,priority,callcode,calldescription,category,call_timestamp,precinct_sca,respondingunit,officerinitiated,intaketime,dispatchtime,traveltime,totalresponsetime,time_on_scene,totaltime,neighborhood,block_id,council_district,longitude,latitude,oid
2917265,202012204585,DPD,Lycaste St & Freud St,48214,3,875030,HANGUP CALLS,HNGUP,2020/05/02 00:54:11+00,0511,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,Conner Creek Industrial,2.616351e+14,5.0,-82.963828,42.362130,458329705
2917266,202012204587,DPD,Iowa St & Sherwood St,48212,3,935030,REMARKS,REMARKS,2020/05/02 00:54:49+00,119C,,Yes,0,0,0,0,0.3,0.3,Grant,2.616351e+14,3.0,-83.033673,42.424966,458329706
2917267,202012204589,DPD,Hayes St & Maddelein St,48205,3,W8190,TOWING DETAIL,TOW,2020/05/02 00:55:32+00,0903,,Yes,0,0,0,0,0.1,0.1,Regent Park,2.616350e+14,3.0,-82.965509,42.436286,458329707
2917268,202012204591,DPD,Chestnut St & Saint Aubin St,48207,3,875030,HANGUP CALLS,HNGUP,2020/05/02 00:55:53+00,0709,NaN,No,0.5,NaN,NaN,NaN,NaN,NaN,Elmwood Park,2.616352e+14,5.0,-83.033629,42.344962,458329708
2917269,202012204603,DPD,E Davison St & Conant St,48212,2,TS,TRAFFIC STOP,TRF STOP,2020/05/02 00:58:51+00,111A,ECT5217,Yes,0,0,0,0,1.3,1.4,Davison,2.616351e+14,3.0,-83.061491,42.415529,458329709


In [5]:
df.describe()

,incident_id,block_id,council_district,longitude,latitude,oid
count,2.917270e+06,2.812511e+06,2.775019e+06,2.917270e+06,2.917270e+06,2.917270e+06
mean,2.018458e+11,2.616335e+14,4.274719e+00,-8.310572e+01,4.237999e+01,4.568711e+08
std,1.031118e+08,3.259691e+10,1.958770e+00,1.122942e-01,4.465901e-02,8.421435e+05
min,2.016264e+11,2.609923e+14,1.000000e+00,-8.413221e+01,4.208159e+01,4.554124e+08
25%,2.017351e+11,2.616351e+14,3.000000e+00,-8.317841e+01,4.235224e+01,4.561418e+08
50%,2.018318e+11,2.616352e+14,5.000000e+00,-8.309684e+01,4.237985e+01,4.568711e+08
75%,2.019228e+11,2.616354e+14,6.000000e+00,-8.303613e+01,4.241668e+01,4.576004e+08
max,2.020122e+11,2.616399e+14,7.000000e+00,-8.271078e+01,4.268528e+01,4.583297e+08


In [6]:
df.shape

(2917270, 24)

### The following commented out sections are no longer necessary

When I found time to come back to this project and give it the appropriate refactor I had noticed
that the administer of the data had corrected the issue caused by this, and by doing so confirmed 
my conclusions. I kept the sections as comments as reference to a unique approach I may wish to 
reference later

One thing that immediately stands out is it seems that we may have two sets of columns representing the latitude and longitude (['X', 'Y'] and ['longitude', 'latitude']). So this will be the first area that we investigate.

The first course of action is to check whether or not the columns are duplicates of eachother.

```Python
geo_df = df[['X', 'Y', 'longitude', 'latitude']]
geo_df.head()
```

```Python
geo_df.loc[geo_df['X'] == geo_df['longitude']]
```

### So they are not exact duplicates which is surprising looking at the first five observations above so what exactly is the difference between them? 

We don't necessarily need to calculate the difference on the entire data set. Instead we will sample the dataset using 26795 observations, enough to ensure a 99% confidence level with 1% margin of error. Then from this sample we will calculate the difference between the corresponding columns and report the mean. This should give us a good idea about the magnitude of the difference. An accuracy of 10 meters will be more than sufficient for this project so if the mean difference is less than .0001 we can feel comfortable in moving forward with one set or the other.

```Python
geo_sample = geo_df.sample(26795, random_state=42)
assert geo_sample.shape[0] == 26795
geo_sample.head()
```

```Python
X_diff = np.mean(abs
                 (df['X'] 
                - df['longitude']))

Y_diff = np.mean(abs
                 (df['Y'] 
                - df['latitude']))

(X_diff,
 Y_diff)
```

```Python
# microns it is, lets go ahead and drop latitude and longitude in favor of X and Y

df = df.drop(columns=['latitude', 'longitude'])
df.head()
```

---
End of deprecated cells
---
---

#### Next order of business is to address the mixed data types warning we received while loading in our data

In [7]:
mixed_types = df.iloc[:,range(14,20)]
mixed_types

,traveltime,totalresponsetime,time_on_scene,totaltime,neighborhood,block_id
0,15.5,546.6,120.5,667.1,Wade,2.616350e+14
1,6.4,14.5,59.1,73.6,Tri-Point,2.616354e+14
2,NaN,NaN,NaN,NaN,Oak Grove,2.616354e+14
3,7.6,15.0,41.7,56.7,We Care Community,2.616354e+14
4,NaN,NaN,NaN,"1,668.4",Barton-McFarland,2.616354e+14
...,...,...,...,...,...,...
2917265,NaN,NaN,NaN,NaN,Conner Creek Industrial,2.616351e+14
2917266,0,0,0.3,0.3,Grant,2.616351e+14
2917267,0,0,0.1,0.1,Regent Park,2.616350e+14
2917268,NaN,NaN,NaN,NaN,Elmwood Park,2.616352e+14


```
mixed_types['intaketime'].astype(float)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-13-bab3625ef491> in <module>
----> 1 mixed_types['intaketime'].astype(float)

~/res/miniconda3/envs/p911-dev/lib/python3.8/site-packages/pandas/core/generic.py in astype(self, dtype, copy, errors)
   5696         else:
   5697             # else, only a single dtype is given
-> 5698             new_data = self._data.astype(dtype=dtype, copy=copy, errors=errors)
   5699             return self._constructor(new_data).__finalize__(self)
   5700 

~/res/miniconda3/envs/p911-dev/lib/python3.8/site-packages/pandas/core/internals/managers.py in astype(self, dtype, copy, errors)
    580 
    581     def astype(self, dtype, copy: bool = False, errors: str = "raise"):
--> 582         return self.apply("astype", dtype=dtype, copy=copy, errors=errors)
    583 
    584     def convert(self, **kwargs):

~/res/miniconda3/envs/p911-dev/lib/python3.8/site-packages/pandas/core/internals/managers.py in apply(self, f, filter, **kwargs)
    440                 applied = b.apply(f, **kwargs)
    441             else:
--> 442                 applied = getattr(b, f)(**kwargs)
    443             result_blocks = _extend_blocks(applied, result_blocks)
    444 

~/res/miniconda3/envs/p911-dev/lib/python3.8/site-packages/pandas/core/internals/blocks.py in astype(self, dtype, copy, errors)
    623             vals1d = values.ravel()
    624             try:
--> 625                 values = astype_nansafe(vals1d, dtype, copy=True)
    626             except (ValueError, TypeError):
    627                 # e.g. astype_nansafe can fail on object-dtype of strings

~/res/miniconda3/envs/p911-dev/lib/python3.8/site-packages/pandas/core/dtypes/cast.py in astype_nansafe(arr, dtype, copy, skipna)
    895     if copy or is_object_dtype(arr) or is_object_dtype(dtype):
    896         # Explicit copy, or required since NumPy can't view from / to object.
--> 897         return arr.astype(dtype, copy=True)
    898 
    899     return arr.view(dtype)

ValueError: could not convert string to float: '4,092.9'
```

We can see here that some of the observations use commas in their formatting so I think the best way for us to take care of this issue would be to define a function to iterate over the columns with mixed types return only numbers and the decimal and then convert the column to numeric type.

In [8]:
def fix_mixed_types(df: pd.DataFrame, cols: list)-> pd.DataFrame:
    """
    Fixes columns with mixed types using regular expressions and pd.to_numeric
    """
    df = df.copy()
    
    for col in cols:
        #    cast column to string type then use regular expressions to remove commas
        df[col] = df[col].astype(str).apply(lambda x: re.sub(r'[^0-9^.]', '', x))
        
        #    cast the reformatted column to float
        df[col] = pd.to_numeric(df[col])
    return df

In [9]:
#    Always good practice to test functions out before applying them to the entire dataset

mixed_types = fix_mixed_types(mixed_types, list(mixed_types.columns))

mixed_types

,traveltime,totalresponsetime,time_on_scene,totaltime,neighborhood,block_id
0,15.5,546.6,120.5,667.1,NaN,2.616350e+14
1,6.4,14.5,59.1,73.6,NaN,2.616354e+14
2,NaN,NaN,NaN,NaN,NaN,2.616354e+14
3,7.6,15.0,41.7,56.7,NaN,2.616354e+14
4,NaN,NaN,NaN,1668.4,NaN,2.616354e+14
...,...,...,...,...,...,...
2917265,NaN,NaN,NaN,NaN,NaN,2.616351e+14
2917266,0.0,0.0,0.3,0.3,NaN,2.616351e+14
2917267,0.0,0.0,0.1,0.1,NaN,2.616350e+14
2917268,NaN,NaN,NaN,NaN,NaN,2.616352e+14


In [10]:
mixed_types.dtypes

traveltime           float64
totalresponsetime    float64
time_on_scene        float64
totaltime            float64
neighborhood         float64
block_id             float64
dtype: object

In [11]:
df = fix_mixed_types(df, list(mixed_types.columns))

df.dtypes

incident_id            int64
agency                object
incident_address      object
zip_code              object
priority              object
callcode              object
calldescription       object
category              object
call_timestamp        object
precinct_sca          object
respondingunit        object
officerinitiated      object
intaketime            object
dispatchtime          object
traveltime           float64
totalresponsetime    float64
time_on_scene        float64
totaltime            float64
neighborhood         float64
block_id             float64
council_district     float64
longitude            float64
latitude             float64
oid                    int64
dtype: object

In [12]:
#    lets also ensure that all our null values are encoded the same way

df = df.fillna(np.NaN)
df

,incident_id,agency,incident_address,zip_code,priority,callcode,calldescription,category,call_timestamp,precinct_sca,respondingunit,officerinitiated,intaketime,dispatchtime,traveltime,totalresponsetime,time_on_scene,totaltime,neighborhood,block_id,council_district,longitude,latitude,oid
0,201626400002,DPD,Promenade Ave & Roseberry St,48213,3,825030,DISTURBANCE,DISTURB,2016/09/20 09:36:12+00,910A,NaN,No,2.2,528.6,15.5,546.6,120.5,667.1,NaN,2.616350e+14,4.0,-82.986962,42.408358,455412440
1,201626400019,DPD,Ardmore St & W 7 Mile Rd,48235,2,811020,AUTO X UNK INJ / IMPAIRED,ACCUNK,2016/09/20 09:46:32+00,1201,NaN,No,2.7,5.2,6.4,14.5,59.1,73.6,NaN,2.616354e+14,2.0,-83.185213,42.430914,455412441
2,201626400020,DPD,Pembroke Ave & Prairie St,48221,3,374030,RECOVER AUTO,RECAUTO,2016/09/20 09:48:24+00,123B,NaN,No,4.9,NaN,NaN,NaN,NaN,NaN,NaN,2.616354e+14,2.0,-83.146057,42.438770,455412442
3,201626400024,DPD,Joy Rd & Robson St,48228,3,825030,DISTURBANCE,DISTURB,2016/09/20 09:50:29+00,0207,NaN,No,1.4,3.4,7.6,15.0,41.7,56.7,NaN,2.616354e+14,7.0,-83.191603,42.358257,455412443
4,201626400029,DPD,Littlefield St & Ellis St,48228,3,374030,RECOVER AUTO,RECAUTO,2016/09/20 09:54:37+00,28A,NaN,No,3,"1,645.3",NaN,NaN,NaN,1668.4,NaN,2.616354e+14,7.0,-83.175124,42.360867,455412444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2917265,202012204585,DPD,Lycaste St & Freud St,48214,3,875030,HANGUP CALLS,HNGUP,2020/05/02 00:54:11+00,0511,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.616351e+14,5.0,-82.963828,42.362130,458329705
2917266,202012204587,DPD,Iowa St & Sherwood St,48212,3,935030,REMARKS,REMARKS,2020/05/02 00:54:49+00,119C,,Yes,0,0,0.0,0.0,0.3,0.3,NaN,2.616351e+14,3.0,-83.033673,42.424966,458329706
2917267,202012204589,DPD,Hayes St & Maddelein St,48205,3,W8190,TOWING DETAIL,TOW,2020/05/02 00:55:32+00,0903,,Yes,0,0,0.0,0.0,0.1,0.1,NaN,2.616350e+14,3.0,-82.965509,42.436286,458329707
2917268,202012204591,DPD,Chestnut St & Saint Aubin St,48207,3,875030,HANGUP CALLS,HNGUP,2020/05/02 00:55:53+00,0709,NaN,No,0.5,NaN,NaN,NaN,NaN,NaN,NaN,2.616352e+14,5.0,-83.033629,42.344962,458329708


In [ ]:
# and finally lets just take a quick look at the cardinality of our categorical columns

df.select_dtypes(exclude='number').nunique().sort_values(ascending=False)

In [14]:
df.head()

,incident_id,agency,incident_address,zip_code,priority,callcode,calldescription,category,call_timestamp,precinct_sca,respondingunit,officerinitiated,intaketime,dispatchtime,traveltime,totalresponsetime,time_on_scene,totaltime,neighborhood,block_id,council_district,longitude,latitude,oid
0,201626400002,DPD,Promenade Ave & Roseberry St,48213,3,825030,DISTURBANCE,DISTURB,2016/09/20 09:36:12+00,910A,NaN,No,2.2,528.6,15.5,546.6,120.5,667.1,NaN,2.616350e+14,4.0,-82.986962,42.408358,455412440
1,201626400019,DPD,Ardmore St & W 7 Mile Rd,48235,2,811020,AUTO X UNK INJ / IMPAIRED,ACCUNK,2016/09/20 09:46:32+00,1201,NaN,No,2.7,5.2,6.4,14.5,59.1,73.6,NaN,2.616354e+14,2.0,-83.185213,42.430914,455412441
2,201626400020,DPD,Pembroke Ave & Prairie St,48221,3,374030,RECOVER AUTO,RECAUTO,2016/09/20 09:48:24+00,123B,NaN,No,4.9,NaN,NaN,NaN,NaN,NaN,NaN,2.616354e+14,2.0,-83.146057,42.438770,455412442
3,201626400024,DPD,Joy Rd & Robson St,48228,3,825030,DISTURBANCE,DISTURB,2016/09/20 09:50:29+00,0207,NaN,No,1.4,3.4,7.6,15.0,41.7,56.7,NaN,2.616354e+14,7.0,-83.191603,42.358257,455412443
4,201626400029,DPD,Littlefield St & Ellis St,48228,3,374030,RECOVER AUTO,RECAUTO,2016/09/20 09:54:37+00,28A,NaN,No,3,"1,645.3",NaN,NaN,NaN,1668.4,NaN,2.616354e+14,7.0,-83.175124,42.360867,455412444


### Lets start filtering down the data set now as we begin work on the core of our model

Not every column will be useful here, at least for modeling purposes, first and foremost we can retain the incident_id column
as this is a unique identifier which we can later use to connect incidents with the rests of the data if we see fit.

In [17]:
model_cols = [
    'incident_id',
    'calldescription',
    'call_timestamp',
    'longitude',
    'latitude'
]

df2 = df[model_cols].rename(columns={'calldescription':'call_description'})
df2.head()

,incident_id,call_description,call_timestamp,longitude,latitude
0,201626400002,DISTURBANCE,2016/09/20 09:36:12+00,-82.986962,42.408358
1,201626400019,AUTO X UNK INJ / IMPAIRED,2016/09/20 09:46:32+00,-83.185213,42.430914
2,201626400020,RECOVER AUTO,2016/09/20 09:48:24+00,-83.146057,42.438770
3,201626400024,DISTURBANCE,2016/09/20 09:50:29+00,-83.191603,42.358257
4,201626400029,RECOVER AUTO,2016/09/20 09:54:37+00,-83.175124,42.360867


In [19]:
# This should be fairly obvious here
df2 = df2.set_index('incident_id')
df2.head(3)

,call_description,call_timestamp,longitude,latitude
incident_id,,,,
201626400002,DISTURBANCE,2016/09/20 09:36:12+00,-82.986962,42.408358
201626400019,AUTO X UNK INJ / IMPAIRED,2016/09/20 09:46:32+00,-83.185213,42.430914
201626400020,RECOVER AUTO,2016/09/20 09:48:24+00,-83.146057,42.438770


In [ ]:
df.

In [20]:
df2.call_description.value_counts()

administrative_calls = [
    "START OF SHIFT INFORMATION",
    "REMARKS",
    "EMPLOYEE CALL IN / TIME OFF"
]





SPECIAL ATTENTION                 455355
TRAFFIC STOP                      403855
DISTURBANCE                       198109
START OF SHIFT INFORMATION        166751
REMARKS                           148500
INVESTIGATE PERSON                109462
UNKNOWN PROBLEM                   100857
HANGUP CALLS                       97167
TOWING DETAIL                      79282
ASSAULT AND BATTERY                72264
FELONIOUS ASSAULT IP               49349
BUS BOARDING                       48870
LARCENY REPORT                     42780
AUTO X UNK INJ / IMPAIRED          37409
VERIFIED ALR / PERSON W/O CODE     29988
UDAA REPORT                        29537
PERSON WITH WEAPON                 27552
INVESTIGATE AUTO                   26594
BURGLARY OCCUPIED RESD I/P         26429
SHOTS FIRED IP                     26317
DV A/B I/P-J/H                     21520
BUILDING CHECK                     20178
MALICIOUS DESTRUCTION I/P          20143
ONE DOWN OR OVER THE WHEEL         19461
MISCELLANEOUS TR